In [ ]:
# Inizializzazione (CPU 2:00 min, GPU 1:50 min)
!git clone https://github.com/abewley/sort.git
!pip install lapx filterpy
! pip install matplotlib==3.2.1

Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 208 (delta 2), reused 2 (delta 1), pack-reused 202 (from 1)
Receiving objects: 100% (208/208), 1.20 MiB | 20.56 MiB/s, done.
Resolving deltas: 100% (74/74), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628944 sha256=e941cdcec2f6e2228a5c316018a14bb4f1f5ef1f29ab7d764db7cdc48816fd64
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=751084f04213b0c3a3fedff6f512fb17ca2b233ae51f89d9e05988a6262fb12a
  Stor

In [ ]:
# Installa YOLO8
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/112.6 GB disk)


In [ ]:
# Importa da Drive il necessario
from google.colab import drive
import os
drive.mount('/content/drive')

#dataset_folder = '/content/drive/MyDrive/CV project/dataset/'
dataset_folder = '/content/drive/MyDrive/Simone/CV project/dataset/'

#os.chdir(dataset_folder)

Mounted at /content/drive


In [ ]:
import matplotlib

In [ ]:
import cv2
import numpy as np
import csv
import json
import os
import sys

#import torch

from ultralytics import YOLO

from tqdm.notebook import tqdm

matplotlib.use('TKAgg')
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('sort'))
os.chdir('/content/sort')

vehicle_classes = {
    'articulated_truck': 0,
    'bus': 2,
    'car': 3,
    'motorcycle': 4,
    'motorized_vehicle': 5,
    'pickup_truck': 8,
    'single_unit_truck': 9,
    'work_van': 10
}

In [ ]:
from sort import Sort

In [ ]:
# Per evitare fastidiosi output durante l'utilizzo di YOLO
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def export_results(folder_path, name, d):
    with open(folder_path + name + ".json", 'w') as f:
       f.write(json.dumps(d))

In [ ]:
# YOLO
def load_yolo_model(weights_path):
    model = YOLO(weights_path)
    return model

def detect_objects_in_frame(model, frame, classes):
    results = model(frame, verbose=False)

    class_ids = []
    confidences = []
    boxes = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf.cpu().numpy().item()
            cls = box.cls.cpu().numpy().item()

            if conf > 0.5 and cls in list(classes.values()):
                boxes.append([x1, y1, x2, y2])
                confidences.append(conf)
                class_ids.append(cls)

    return boxes, class_ids, confidences

In [ ]:
# Da ogni frame ottieni dati dei veicoli
def get_vehicles_position(model, tracker, frame, mask):

    # Applica YOLO
    roi_frame = cv2.bitwise_and(frame, frame, mask=mask)
    boxes, classes, confs = detect_objects_in_frame(model, roi_frame, vehicle_classes)
    if len(boxes) != 0:
        tracked_objects = tracker.update(np.array(boxes))
    else:
        return {}

    objects = {}
    for obj, cls, conf in zip(tracked_objects, classes, confs):
        x1, y1, x2, y2, obj_id = obj
        objects[obj_id] = {"id": obj_id, "class": cls, "bbox": [x1, y1, x2, y2], "conf": conf}

    return objects

In [ ]:
# MAIN
#folder_path = dataset_folder + "publishing_data/Videos/"
folder_path = "/content/drive/MyDrive/Simone/publishing_data/Videos/"
frame_rate = 120
mask_path = dataset_folder + "_ROI_one.png"
weights_yolo8 = dataset_folder + "../weights_yolo8/train/weights/best.pt"
output_folder = dataset_folder + "results_yolo8_mio/"

model = load_yolo_model(weights_yolo8)
tracker = Sort()

# Importa maschera
roi_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask = cv2.threshold(roi_mask, 127, 255, cv2.THRESH_BINARY)[1]


# Lista video elaborati e da elaborare
videos_list = sorted([f for f in os.listdir(folder_path) if f.lower().endswith('.mov')])

# Inizializza csv di output
exporting_data = []

fatti = 0
for video_file in videos_list:
    if fatti < 134:
        fatti += 1
        continue
    video_path = os.path.join(folder_path, video_file)
    print(f"Elaborazione {video_path}")
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number = 1

    history = {}
    while cap.isOpened():
        if frame_number % 120 == 0:
            print(f"{frame_number}/{total_frames}\t{frame_number // 120} s")
        ret, frame = cap.read()
        if not ret:
            break

        history[frame_number] = get_vehicles_position(model, tracker, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), mask)

        frame_number += 1

    cap.release()

    export_results(output_folder, os.path.splitext(video_file)[0], history)
    #break   # fa solo il primo video

Elaborazione /content/drive/MyDrive/Simone/publishing_data/Videos/IMG_0640.MOV


WARNING ⚠️ NMS time limit 0.550s exceeded


120/701	1 s
240/701	2 s
360/701	3 s
480/701	4 s
600/701	5 s
Elaborazione /content/drive/MyDrive/Simone/publishing_data/Videos/IMG_0641.MOV
120/3739	1 s
240/3739	2 s
360/3739	3 s
480/3739	4 s
600/3739	5 s
720/3739	6 s
840/3739	7 s
960/3739	8 s
1080/3739	9 s
1200/3739	10 s
1320/3739	11 s
1440/3739	12 s
1560/3739	13 s
1680/3739	14 s
1800/3739	15 s
1920/3739	16 s
2040/3739	17 s
2160/3739	18 s
2280/3739	19 s
2400/3739	20 s
2520/3739	21 s
2640/3739	22 s
2760/3739	23 s
2880/3739	24 s
3000/3739	25 s
3120/3739	26 s
3240/3739	27 s
3360/3739	28 s
3480/3739	29 s
3600/3739	30 s
3720/3739	31 s
Elaborazione /content/drive/MyDrive/Simone/publishing_data/Videos/IMG_0642.MOV
120/11900	1 s
240/11900	2 s
360/11900	3 s
480/11900	4 s
600/11900	5 s
720/11900	6 s
840/11900	7 s
960/11900	8 s
1080/11900	9 s
1200/11900	10 s
1320/11900	11 s
1440/11900	12 s
1560/11900	13 s
1680/11900	14 s
1800/11900	15 s
1920/11900	16 s
2040/11900	17 s
2160/11900	18 s
2280/11900	19 s
2400/11900	20 s
2520/11900	21 s
2640/11900	22 s

In [ ]:

export_results(output_folder, os.path.splitext(video_file)[0], history)